Code to make h5 files of the templates in the case that they are not saved correctly during make_templates

Created December 7, 2022
Updated May 15, 2023

In [2]:
import pandas as pd #to work with csv and dataframe
import numpy as np #for array
from eqcorrscan import Tribe #import the ability to read .tgz files with templates
import h5py #for writing to h5

Matched-filter CPU is not compiled! Should be here: /home/jupyter_share/miniconda3/envs/seismo/lib/python3.8/site-packages/fast_matched_filter/lib/matched_filter_CPU.so


** To Delete **

Picking out good template waveforms

In [1]:
from glob import glob
from obspy import Trace
homedir = '/home/smocz/expand_redpy_new_files/templates/' #directory to with .tgz file
savedir = '/home/smocz/expand_redpy_new_files/h5/' #directory to save h5 file in
h5name = 'curated_set.h5'

In [6]:
#find t in T to save; browsing
volc = "Rainier"
sta = "VOIT"
filenamepath = glob(f"{homedir}Volcano_{volc}_Network_*_Station_{sta}_Channel_*.tgz")[0]

# print(filename)

T = Tribe().read(f'{filenamepath}')

for t in T:
    print(t.name) #template name
    t.st.plot(); #tempalte stream
#     break

NameError: name 'sta' is not defined

In [3]:
#save t to wave & name list
volc = "Rainier"
#list of templates to get streams of
curated_t_name_list = ['rcmhhzrpra013','rcmhhzrpra088','kautbhzrpra696','kautbhzrpra719','rcmhhzrpra368', \
                       'carbbhzrpra368','carbbhzrpra413','rcmhhzrpra295','voitbhzrpra018','voitbhzrpra063', \
                       'rcmhhzrpra283',"carbbhzrpra253","carbbhzrpra357","carbbhzrpra907"]
group_id_list = [0,0,0,0,1,1,1,2,2,2,3,3,3,3]
sta_list = ["VOIT", 'RCM','KAUT','CARB']

wave_list = []
name_list = []
for name in curated_t_name_list:
    for sta in sta_list:
        if name.startswith(sta.lower()):
            filenamepath = glob(f"{homedir}Volcano_{volc}_Network_*_Station_{sta}_Channel_*.tgz")[0]
            T = Tribe().read(f'{filenamepath}')
            for t in T:
                if name == t.name:
                    t_name = t.name
#                     print(t.name)
                    wave = t.st
#                     t.st.plot();
                    wave_list.append(np.array(t.st[0].data)) #append stream data as an array
                    name_list.append(t.name) #append names to list
                else:
                    continue
        else:
            continue

wave_arr = np.array(wave_list)
print(wave_arr.shape) #should be (# of streams, length of streams) like (90, 3121) for 90 streams of length 3121

(14, 3121)


In [4]:
with h5py.File(f"{savedir}{h5name}", "w") as f:
    f.create_dataset("waveforms", data=wave_arr)
    f.create_dataset("group_id", data=group_id_list)
    f.create_dataset("template_name", data=name_list)

Writing H5

In [ ]:
homedir = '/home/smocz/expand_redpy_new_files/templates/' #directory to with .tgz file
filename = 'Volcano_Rainier_Network_UW_Station_RCM_Channel_HHZ.tgz' #name of .tgz file
savedir = '/home/smocz/expand_redpy_new_files/h5/' #directory to save h5 file in
h5name = 'selected_waves.h5' #name of h5 file

In [ ]:
T = Tribe().read(f'{homedir}{filename}') #read the .tgz file

#create data
wave_list = [] #list of arrays of stream data
name_list = [] #list of template names

for t in T[:20]: #for each of the first 20 templates in the tgz file
    t_id = t.name #template name
    waveform = t.st #tempalte stream
    wave_list.append(np.array(t.st[0].data)) #append stream data as an array
    name_list.append(t.name) #append names to list
#     break
    
wave_arr = np.array(wave_list)
print(wave_arr.shape) #should be (# of streams, length of streams) like (90, 3121) for 90 streams of length 3121

In [ ]:
#add data to h5 file
with h5py.File(f"./{h5name}", "w") as f:
    f.create_dataset("waveforms", data=wave_arr)
    f.create_dataset("template_name", data=name_list)

Reading H5

In [ ]:
#load waveforms from h5 file
with h5py.File("./specufex_data.h5", "r") as f:
    waveforms = f["waveforms"][()]
    template_name = f["template_name"].asstr()[()]

print(f"{len(waveforms)} waveforms in file")

tdf = pd.DataFrame({"template_name":template_name,"waveform":list(waveforms)})
tdf.head()